In [ ]:
from ultralytics import YOLO, SAM
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import utils
import math
import os
import json
import cv2
import glob

In [ ]:
model = YOLO("best.pt")

In [ ]:

path = "datasets/zweite_messung"
results = model.predict(path)
lst = os.listdir(path)
json_data = {}

In [ ]:
def area(mask, angle, distance):
    angleFactor = 1/math.cos(angle*math.pi/180)
    distanceFactor = distance*distance
    correctedHeight = int(mask.shape[0]*angleFactor)
    resized = cv2.resize(mask, dsize=(mask.shape[1], correctedHeight), interpolation=cv2.INTER_CUBIC)
    print("angleFactor", angleFactor, "correctedHeight", correctedHeight, "distanceFactor", distanceFactor)
    return np.sum(resized)* distanceFactor

In [ ]:
for f in lst:
    _, name  = os.path.split(f)
    base, ending = name.split(".")
    number = base.split("_")[0]
    if ending != "json":
        continue
    with open(os.path.join(path, f))as file:
        d = json.load(file)
        json_data[int(number)] = d

yolodata = {}
image_to_rawarea = {}

for result in results:
    biggest = 0
    biggest_a = 0
    for index, (_, _, w, h) in enumerate(result.boxes.xywh):
        a = w*h
        if a > biggest_a:
            biggest = index
            biggest_a = a

    _, name = os.path.split(result.path)
    number = int(name.split("_")[0])
    mask = result.masks.data.cpu().numpy()[biggest, :, :]
    angle = json_data[number]["Angle"]
    distance = json_data[number]["Distance"]
    proxy = area(mask, angle, distance)
    conf = result.boxes.conf.cpu().numpy()[biggest]
    print(name)
    print("raw sum", np.sum(mask))
    print("index", biggest, "conf", conf, "proxy", proxy, "angle", angle, "distance", distance)
    utils.plot(result)
    yolodata[number] = proxy
    image_to_rawarea[number] = np.sum(mask)

In [ ]:
# From what I learned by taking calibration photos 
# at 20cm 1cm is equal to 31.73 pixels
# at 30cm 1cm is equal to 22.6190476190 pixels
# at 2m 1cm is equal to 2.835 pixels
normalized_area1 = 22.6190476190*22.6190476190*0.3*0.3
normalized_area2 = 31.73*31.73*0.2*0.2
normalized_area3 = 2.835*2.835*2*2
print(normalized_area1)
print(normalized_area2)
print(normalized_area3)

In [ ]:
image_to_ohrenmarke={
    4:10610469972,
    5:10610469972,
    6:10610469975,
    7:10610469975,
    8:10610469975,
    9:10610469975,
    10:10610469975,
    11:10610469985,
    12:10610469985,
    13:10610469985,
    14:10610436179,
    15:10610436179,
    16:10610536153,
    17:10610536153,
    18:10610436179,
    19:10610436177,
    20:10610436177,
    21:10610469982,
    22:10610469982,
    23:10610470885,
    24:10610436177,
    25:10610436171,
    26:10610536153,
    27:10610469975,
    28:10610436175,
    29:10610536162,
    30:10610536162,
}
df = pd.read_csv("alle_daten.csv")
df_weight = "Gewicht"
df_length = "Länge"
df_hg = "Brustumfang"
df_shoulder = "Schulterhöhe"
df_rump = "Kreuzbeinhöhe"
df_number = "Ohrmarkennummer"
print(df)
print(image_to_rawarea)
print(json_data)

In [ ]:
areas = []
distances = []
raw_areas = []
weights = []
numbers = []
for key, area in image_to_rawarea.items():
    if key in [11, 12, 13]: # we dont have the data for these pictures
        continue 
    weight = df[df[df_number] == image_to_ohrenmarke[key]][df_weight].values[0]
    weights.append(weight)
    areas.append(area/weight)
    raw_areas.append(area)
    distances.append(json_data[key]["Distance"])
    numbers.append(key)

proxy = np.array(raw_areas) * np.array(distances)*np.array(distances)
plt.scatter(distances, areas)
plt.show()
plt.scatter(weights, proxy)
for i, label in enumerate(image_to_rawarea.keys()):
    plt.text(weights[i], proxy[i], label, fontsize=12, ha='right', va='bottom')

In [ ]:
# pearson cooefficient for the area for erste messung
# there is no correction for the angle done since data was not available
from scipy.stats import pearsonr
first_weights = [65.9,82.3,60,74.5,34.8,]
second_weights = [67.3,58,39.4,72.5,37.1,30.7,]
weights = first_weights + second_weights
first_lengths = [85,88,85,94,70,]
second_lengths = [83,81,71,88,67,65,]
lengths = first_lengths + second_lengths
first_hg = [98,105,92,104,79,]
second_hg = [98,92,80,101,78,80]
hg = first_hg + second_hg
first_sacrum = [73,75,72,77,63,]
second_sacrum = [74,72,69,77,67,62]
sacrum = first_sacrum + second_sacrum
first_shoulder = [74,75,72,79,62,]
second_shoulder = [70,69,65,79,63,60]
shoulder = first_shoulder + second_shoulder
first_areas = [126217.6013,157085.518,146242.3731,192738.8976,95095.41843,]
second_areas = [239956.2573,256128.978,192941.0119,272832.2422,171866.9452,161795.5374,]
areas = first_areas + second_areas

area_result = pearsonr(areas, weights)
print("area","r", area_result.correlation, "p", area_result.pvalue)
plt.scatter(weights, areas)
first_area_result = pearsonr(first_areas, first_weights)
print("first_area","r", first_area_result.correlation, "p", first_area_result.pvalue)
plt.scatter(first_weights, first_areas)
second_area_result = pearsonr(second_areas, second_weights)
print("second_area","r", second_area_result.correlation, "p", second_area_result.pvalue)
plt.scatter(second_weights, second_areas)
hg_result = pearsonr(hg, weights)
print("hg","r", hg_result.correlation, "p", hg_result.pvalue)
shoulder_result = pearsonr(shoulder, weights)
print("shoulder", "r", shoulder_result.correlation, "p", shoulder_result.pvalue)
sacrum_result = pearsonr(sacrum, weights)
print("sacrum", "r", sacrum_result.correlation, "p", sacrum_result.pvalue)
length_result = pearsonr(lengths, weights)
print("length", "r", length_result.correlation, "p", length_result.pvalue)